In [1]:
import cv2
import mahotas
from keras.models import load_model

from pyimagesearch.preprocessing.imagetoarraypreprocessor import ImageToArrayPreprocessor
from pyimagesearch.preprocessing.simplepreprocessor import SimplePreprocessor
from pyimagesearch.datasets.simpledatasetloader import SimpleDatasetLoader
from keras.preprocessing.image import img_to_array
import numpy as np



Using TensorFlow backend.
/home/autom8-sudesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/autom8-sudesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/autom8-sudesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/autom8-sudesh/.local/lib/python3.6/sit

In [2]:
print("[INFO] loading pre-trained network...")
model = load_model('MNISTMODEL2.hdf5')

[INFO] loading pre-trained network...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:
classLabels = ["0","1","2","3","4","5","6","7","8","9"]

In [26]:
imagePath="/home/autom8-sudesh/Desktop/images.jpeg"
image = cv2.imread(imagePath)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray,(5,5),0)
edged=cv2.Canny(blurred,30,150)

(cnts,_) = cv2.findContours(edged.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)


#cnts = sorted([c,cv2.boundingRect(c)[0] for c in cnts], key = lambda x:x[1])

for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    
    if w>=1 and h>= 1:
        roi = gray[y-5:y + h+5, x-5:x + w+5]
        thresh = roi.copy()
        T = mahotas.thresholding.otsu(roi)
        thresh[thresh > T] = 255
        thresh = cv2.bitwise_not(thresh)
        thresh=cv2.resize(thresh, (28,28),cv2.INTER_AREA)
       
        
        thresh = img_to_array(thresh)
       
        thresh = thresh.astype("float") / 255.0
        
        digit = model.predict(thresh.reshape(-1,28,28,1),batch_size=1).argmax(axis=1)[0]
        digit=str(digit)
        
       
        cv2.rectangle(image, (x-5, y-5), (x + w+5, y + h+5),(0, 255, 0), 1)
        cv2.putText(image, str(digit), (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("Recognized_digit5.jpg",image)

True